In [2]:
import torch
x = torch.tensor([2, 3, 4], dtype=torch.float, requires_grad=True)
print(x)
y = x * 2
while y.norm() < 1000:
    y = y * 2
print(y)
y.backward(torch.ones_like(y))
print(x.grad)


tensor([2., 3., 4.], requires_grad=True)
tensor([ 512.,  768., 1024.], grad_fn=<MulBackward0>)
tensor([256., 256., 256.])


In [3]:

x = torch.tensor([2,2],dtype=torch.float32,requires_grad=True)

#Case 1
y = x * x
y.sum().backward()
y,x.grad
#Case 2


(tensor([4., 4.], grad_fn=<MulBackward0>), tensor([4., 4.]))

In [7]:
x.grad.zero_()
y1 = torch.dot(x,x)
y1.backward()
y1,x.grad,x.view(1,2)

(tensor(8., grad_fn=<DotBackward0>),
 tensor([4., 4.]),
 tensor([[2., 2.]], grad_fn=<ViewBackward0>))

In [11]:
x = torch.tensor([2., 1.], requires_grad=True)
y = torch.tensor([[1., 2.], [3., 4.]], requires_grad=True)

z = torch.mm(x.view(1, 2), y)
z,x,y,x.view(1,2)

(tensor([[5., 8.]], grad_fn=<MmBackward0>),
 tensor([2., 1.], requires_grad=True),
 tensor([[1., 2.],
         [3., 4.]], requires_grad=True),
 tensor([[2., 1.]], grad_fn=<ViewBackward0>))

In [1]:
clear

In [56]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

In [57]:
def synthetic_data(w,b,num_examples):
    X = torch.normal(0,1,(num_examples,len(w)))
    y = torch.matmul(X,w) + b
    y += torch.normal(0,0.01,y.shape)
    return X,y.reshape((-1,1))

true_w = torch.tensor([-2,-3.4])
true_b = 4.2
features,labels = synthetic_data(true_w,true_b,1000)


In [58]:
# d2l.set_figsize()
# d2l.plt.scatter(features[:,1].detach().numpy(),labels.detach().numpy(),1)

In [59]:
def data_iter(batch_size,features,labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0,num_examples,batch_size):
        batch_indices = torch.tensor(
        indices[i:min(i+batch_size,num_examples)])
        yield features[batch_indices],labels[batch_indices]

In [60]:
batch_size = 20
for X,y in data_iter(batch_size,features,labels):
    print(X,'\n',y)
    break


tensor([[ 3.0382, -0.7230],
        [-0.1171, -0.3877],
        [ 0.9153, -1.4330],
        [-1.3969,  1.2224],
        [ 1.1092, -1.2493],
        [-0.2511, -0.4916],
        [ 0.9761,  1.2399],
        [-0.9159,  0.5976],
        [ 0.4933, -0.2480],
        [-2.5319, -1.2822],
        [-0.3733, -0.8967],
        [-1.2989, -0.1246],
        [-0.1516,  0.5345],
        [ 0.1705,  1.4761],
        [-1.2914,  1.2234],
        [ 1.5859,  0.5187],
        [-0.1911,  0.0207],
        [ 0.6097, -0.5226],
        [ 0.8553,  0.4920],
        [-0.3415,  0.5573]]) 
 tensor([[ 0.5758],
        [ 5.7675],
        [ 7.2473],
        [ 2.8324],
        [ 6.2258],
        [ 6.3617],
        [-1.9613],
        [ 4.0019],
        [ 4.0456],
        [13.6190],
        [ 7.9815],
        [ 7.2063],
        [ 2.6836],
        [-1.1703],
        [ 2.6476],
        [-0.7386],
        [ 4.5167],
        [ 4.7571],
        [ 0.8261],
        [ 2.9989]])


In [69]:
#初始化模型参数
w = torch.normal(0,0.01,size = (2,1),requires_grad=True)
b = torch.zeros(1,requires_grad = True)


In [62]:
#定义模型
def linreg(X,w,b):
    return torch.matmul(X,w) + b

In [63]:
#定义损失函数
def squared_loss(y_hat,y):
    return (y_hat - y.reshape(y_hat.shape)) **2 / 2

In [64]:
#优化函数
def sgd(params,lr,batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()
        

In [70]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss


for epoch in range(num_epochs):
    for X,y in data_iter(batch_size,features,labels):
        l = loss(net(X,w,b),y)
        l.sum().backward()
        sgd([w,b],lr,batch_size)
    with torch.no_grad():
        train_l = loss(net(features,w,b),labels)
        print(f'epoch {epoch +1},loss{float(train_l.mean()):f}')

epoch 1,loss0.849398
epoch 2,loss0.045563
epoch 3,loss0.002487
